# How to Submit an entry to the Competition

### This notebook will show how to
- instantiate dataloader for the demo data
- instantiate pytorch model
- load the model weights
- generate the submission file with the trained model
- upload the submission files to the website

# Step 1) Instantiate the pre-trained model

### Imports

In [1]:
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from nnfabrik.builder import get_data, get_model, get_trainer

### Instantiate DataLoader

In [2]:
# loading the SENSORIUM dataset
filenames = ['../data/static26872-17-20-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip', ]

dataset_fn = 'sensorium.datasets.static_loaders'
dataset_config = {'paths': filenames,
                 'normalize': True,
                 'include_behavior': False,
                 'include_eye_position': False,
                 'batch_size': 128,
                 'scale':1,
                 }

dataloaders = get_data(dataset_fn, dataset_config)

# Instantiate the LN Baseline model

Our LN model has the same architecture as our CNN model (a convolutional core followed by a gaussian readout)
but with all non-linearities removed except the final ELU+1 nonlinearity.
Thus turning the CNN model effectively into a fully linear model followed by a single output non-linearity.


In [3]:
model_fn = 'sensorium.models.stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
              'stack': -1,
              'layers': 3,
              'input_kern': 9,
              'gamma_input': 6.3831,
              'gamma_readout': 0.0076,
              'hidden_kern': 7,
              'hidden_channels': 64,
              'grid_mean_predictor': {'type': 'cortex',
              'input_dimensions': 2,
              'hidden_layers': 1,
              'hidden_features': 30,
              'final_tanh': True},
              'depth_separable': True,
              'init_sigma': 0.1,
              'init_mu_range': 0.3,
              'gauss_type': 'full',
              'linear': True
               }
model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=42,)

# load checkpoints

In [4]:
model.load_state_dict(torch.load("../model_tutorial/model_checkpoints/pretrained/sensorium_ln_model.pth"));

In [5]:
# important: setting the model to evaluation mode
model.eval();

---

# Step 2) Generate the submission file

In [6]:
# specify the dataset_name for the dataloader, in this case, it's the sensorium dataset
dataset_name = '26872-17-20'

# alternatively:
# dataset_name = list(dataloaders["train"].keys())[0]

In [7]:
# import the submission API

from sensorium.utility import submission

In [8]:
# generate the submission file
submission.generate_submission_file(trained_model=model, 
                                    dataloaders=dataloaders,
                                    data_key=dataset_name,
                                    path="./submission_files/",
                                    device="cuda")

Submission file saved for tier: live_test. Saved in: ./submission_files/submission_file_live_test.csv
Submission file saved for tier: final_test. Saved in: ./submission_files/submission_file_final_test.csv


#### That's it.

now you can upload the "live_test"  and "final_test" submission files to the competition homepage. All the metrics will be automatically calculated.

---

In [9]:
import pandas as pd
pd.read_csv('./submission_files/submission_file_live_test.csv')

,trial_indices,image_ids,prediction,neuron_ids
0,cfa0860e83a4c3a763a7e62d825349f7,7017,"[0.22635304927825928, 0.44778627157211304, 0.1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15..."
1,4d5b995358e7798bc7e9d9db83c612a5,7017,"[0.22635304927825928, 0.44778627157211304, 0.1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15..."
2,ef0917ea498b1665ad6c701057155abe,7017,"[0.22635304927825928, 0.44778627157211304, 0.1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15..."
3,331316d4efb44682092a006307b9ae3a,7017,"[0.22635304927825928, 0.44778627157211304, 0.1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15..."
4,d8c24ca8f23c562a5600876ca2a550ce,7017,"[0.22635304927825928, 0.44778627157211304, 0.1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15..."
...,...,...,...,...
985,ce059ef4192cbdcb40df4422c090f1c3,3438,"[0.2734982967376709, 0.2858245372772217, 0.182...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15..."
986,c77cfd5563c8ec4bfcde94c09098ba84,3438,"[0.2734982967376709, 0.2858245372772217, 0.182...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15..."
987,9570efef719d705326f0ff817ef084e6,3438,"[0.2734982967376709, 0.2858245372772217, 0.182...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15..."
988,5faf461eff3099671ad63c6f3f094f7f,3438,"[0.2734982967376709, 0.2858245372772217, 0.182...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15..."


You can of create your own submission file without our API. You'd simply need to create a .csv with the same structure as this one here:
- each row is a particular trial (from dataset/meta/trials/trial_idx.npy)
- each trial has its corresponding image id (from dataset/meta/trials/frame_image_id.npy)
- prediction and neuron_ids are two lists, that contain the predictions for all neurons and their IDs. The predictions and the neuron_ids have to match.

---

# !! Important !!

Our grund truth file is storing **standardized responses**, meaning the responses of each neuron are divided by their own STD. Our dataloader is automatically normalizing the images and responses, and we encourage you to use our DataLoader and our submission API

---

# The submission process

- now that the .csv file is generated, you can make an entry to the competition
- all that is left to do is:
 - register a new user on http://sensorium2022.net/ with a valid email address
 - upload the two .csv files
 - the competition score will be calculated automatically and will show up in the live leaderboard within a few seconds

---